In [1]:
import numpy as np 
import pandas as pd 
import string

import re
import requests
from bs4 import BeautifulSoup

pd.set_option('display.max_colwidth', 100)

import IPython
sound_file = '../../../data/sounds/puzzle_solved_jingle.wav'

In [2]:
df = pd.read_csv("../../../../archive/reviews-1-115.csv")
df.head()

,steamid,appid,voted_up,votes_up,votes_funny,weighted_vote_score,playtime_forever,playtime_at_review,num_games_owned,num_reviews,review,unix_timestamp_created,unix_timestamp_updated
0,76561198107294407,10,True,0,0,0.0,8549,8549,3,1,best gameplay ever created,1619038050,1619038050
1,76561198011733201,10,True,0,0,0.0,25381,25381,74,1,Timeless. Thank you volvo.,1619013754,1619013754
2,76561198168961276,10,True,0,0,0.0,681,616,26,6,this game literally made me love guns and fps game. what else could i say?,1619008681,1619008681
3,76561198957877160,10,True,0,0,0.0,2071,1990,5,1,If this gets a rework in 2021 it's going to be top game for years.,1619003706,1619003706
4,76561199050314447,10,True,0,0,0.0,36718,36718,3,1,ok,1618994842,1618994842


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460621 entries, 0 to 1460620
Data columns (total 13 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   steamid                 1460621 non-null  int64  
 1   appid                   1460621 non-null  int64  
 2   voted_up                1460621 non-null  bool   
 3   votes_up                1460621 non-null  int64  
 4   votes_funny             1460621 non-null  int64  
 5   weighted_vote_score     1460621 non-null  float64
 6   playtime_forever        1460621 non-null  int64  
 7   playtime_at_review      1460621 non-null  int64  
 8   num_games_owned         1460621 non-null  int64  
 9   num_reviews             1460621 non-null  int64  
 10  review                  1458810 non-null  object 
 11  unix_timestamp_created  1460621 non-null  int64  
 12  unix_timestamp_updated  1460621 non-null  int64  
dtypes: bool(1), float64(1), int64(10), object(1)
memory usage

In [4]:
game_ids = list(df["appid"].unique())

url_list = []
for each in game_ids:
    url = "https://store.steampowered.com/app/{}/".format(each)
    url_list.append(url)

In [5]:
def get_title(soup):
    title = soup.find('div', class_="apphub_AppName")
    return title
    
def get_tags(soup):
    warning = soup.find('div', class_="glance_tags popular_tags")
    tags = [p.text for p in warning.findAll('a', class_="app_tag")]
    
    for index in range(len(tags)):
        tags[index] = tags[index].replace("\t", "")
        tags[index] = tags[index].replace("\r\n", "")
    
    return tags

In [6]:
titles = []
tags = []

for url in url_list:
    html_page = requests.get(url)
    soup = BeautifulSoup(html_page.content, 'html.parser')
    titles += get_title(soup)
    tags.append(get_tags(soup))

In [7]:
display(len(game_ids), len(titles), len(tags))

96

96

96

In [10]:
for ind, row in df.iterrows():
    for index in range(len(game_ids)):
        if row["appid"] == game_ids[index]:
            row["app_title"] = titles[index]
            row["app_tags"] = tags[index]

KeyboardInterrupt: 

In [14]:
def temp_func(appid):
    url = "https://store.steampowered.com/app/{}/".format(appid)
    html_page = requests.get(url)
    soup = BeautifulSoup(html_page.content, 'html.parser')
    title = get_title(soup)
    return title

In [ ]:
df['app_title'] = df['appid'].apply(lambda x: temp_func(x))

In [ ]:
IPython.display.Audio(sound_file, autoplay=True, rate=1000)